In [22]:
%run Requirements.ipynb

In [23]:
conn = mysql.connector.connect(
    host='127.0.0.1',
    user='root',
    password='ronitgupta28',
    database='formulaOne'
)
cursor = conn.cursor()

Consolidated Race Results With Drivers and Constructors

In [24]:
df = spark.sql("""SELECT R.RaceID,
    R.Year,
    R.Round,
    R.CircuitID,
    R.Name AS RaceName,
    R.Date AS RaceDate,
    R.Time AS RaceTime,
    D.DriverID,
    D.Points AS DriverPoints,
    RS.Position AS DriverPosition,
    RS.ConstructorID,
    C.Points AS ConstructorPoints,
    C.Position AS ConstructorPosition,
    RS.Laps,
    RS.Grid,
    RS.FastestLap,
    RS.FastestLapTime,
    RS.FastestLapSpeed,
    RS.StatusID,
    CONCAT(R.RaceID,'-',D.DriverID) AS RaceDriverID,
    CONCAT(R.RaceID,'-',D.DriverID,'-',C.ConstructorID) AS RaceDriverConstructorID 
    FROM PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/races` AS R 
LEFT JOIN PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/results` AS RS ON RS.raceId = R.raceid 
INNER JOIN PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/driver_standings` AS D ON R.raceId = D.raceId AND RS.driverid = D.driverid 
LEFT JOIN PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/constructor_standings` AS C ON R.raceId = C.raceId AND RS.constructorid = C.constructorid""")


df.write.mode("overwrite").format('parquet').options(header=True).save('MartData/TotalRaceResults')

In [25]:
import numpy as np
df = pd.read_parquet('/Users/ronitguptaaa/Documents/FormulaOne/MartData/TotalRaceResults')

df.fillna(value=np.nan, inplace=True)

df.replace({np.nan: "NULL"}, inplace=True)
columns = ", ".join([f"`{col}` VARCHAR(255)" for col in df.columns])

drop_table_query = f"DROP TABLE IF EXISTS TotalRaceResults;"
cursor.execute(drop_table_query)
create_table_query = f"CREATE TABLE IF NOT EXISTS TotalRaceResults ({columns});"
cursor.execute(create_table_query)

for index, row in df.iterrows():
    values = tuple(row.values)
    insert_query = f"REPLACE INTO TotalRaceResults VALUES {values};"
    cursor.execute(insert_query)

conn.commit()

In [26]:
df = pd.read_parquet('/Users/ronitguptaaa/Documents/FormulaOne/MartData/TotalRaceResults')

columns = ", ".join([f"`{col}` VARCHAR(255)" for col in df.columns])

drop_table_query = f"DROP VIEW IF EXISTS vw_TotalRaceResults;"
cursor.execute(drop_table_query)
create_table_query = f"""CREATE VIEW vw_TotalRaceResults AS (SELECT * FROM TotalRaceResults);"""
cursor.execute(create_table_query)

conn.commit()

Driver Details

In [27]:
df = spark.sql(
    """SELECT 
    CAST(driverid AS INT) AS DriverID,
    CAST(CONCAT(forename,' ',surname) AS STRING) AS DriverName,
    dob AS DateOfBirth,
    CAST(nationality AS STRING) AS Nationality
    FROM PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/drivers`"""
)

df.write.mode('overwrite').format('parquet').options(header=True).save('MartData/DriverDetails')

In [28]:
df = pd.read_parquet("/Users/ronitguptaaa/Documents/FormulaOne/MartData/DriverDetails")

columns = ", ".join([f"`{col}` VARCHAR(255)" for col in df.columns])

drop_table_query = f"DROP TABLE IF EXISTS DriverDetails;"
cursor.execute(drop_table_query)
create_table_query = f"CREATE TABLE IF NOT EXISTS DriverDetails ({columns});"
cursor.execute(create_table_query)

for index, row in df.iterrows():
    values = tuple(row.values)
    insert_query = f"REPLACE INTO DriverDetails VALUES {values};"
    cursor.execute(insert_query)

conn.commit()

In [29]:
df = pd.read_parquet("/Users/ronitguptaaa/Documents/FormulaOne/MartData/DriverDetails")

columns = ", ".join([f"`{col}` VARCHAR(255)" for col in df.columns])

drop_table_query = f"DROP VIEW IF EXISTS vw_DriverDetails;"
cursor.execute(drop_table_query)
create_table_query = f"CREATE VIEW vw_DriverDetails AS (SELECT * FROM DriverDetails);"
cursor.execute(create_table_query)


conn.commit()

Constructor Details

In [30]:
df = spark.sql(
    """SELECT 
        CAST(constructorid AS INT) AS ConstructorID,
        CAST(name AS STRING) AS ConstructorName,
        CAST(nationality AS STRING) AS Nationality
    FROM PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/constructors`"""
)

df.write.mode('overwrite').format('parquet').options(header=True).save('MartData/ConstructorDetails')

In [31]:
df = pd.read_parquet("/Users/ronitguptaaa/Documents/FormulaOne/MartData/ConstructorDetails")

columns = ", ".join([f"`{col}` VARCHAR(255)" for col in df.columns])

drop_table_query = f"DROP TABLE IF EXISTS ConstructorDetails;"
cursor.execute(drop_table_query)
create_table_query = f"CREATE TABLE IF NOT EXISTS ConstructorDetails ({columns});"
cursor.execute(create_table_query)

for index, row in df.iterrows():
    values = tuple(row.values)
    insert_query = f"REPLACE INTO ConstructorDetails VALUES {values};"
    cursor.execute(insert_query)

conn.commit()

In [32]:
df = pd.read_parquet("/Users/ronitguptaaa/Documents/FormulaOne/MartData/ConstructorDetails")

columns = ", ".join([f"`{col}` VARCHAR(255)" for col in df.columns])

drop_table_query = f"DROP VIEW IF EXISTS vw_ConstructorDetails;"
cursor.execute(drop_table_query)
create_table_query = f"CREATE VIEW vw_ConstructorDetails AS (SELECT * FROM ConstructorDetails);"
cursor.execute(create_table_query)

conn.commit()

Circuits Details

In [33]:
df = spark.sql(
    """SELECT 
        CAST(circuitid AS INT) AS CircuitID,
        CAST(name AS STRING) AS CircuitName,
        CAST(location AS STRING) AS CircuitLocation,
        CAST(country AS STRING) AS Country
    FROM PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/circuits`"""
)

df.write.mode('overwrite').format('parquet').options(header=True).save('MartData/CircuitDetails')

In [34]:
from pyspark.sql.functions import when

df = spark.read.parquet("/Users/ronitguptaaa/Documents/FormulaOne/MartData/CircuitDetails")

# Perform the update using DataFrame transformations
df = df.withColumn("Country", when(df["Country"] == "United States", "USA").otherwise(df["Country"]))


In [35]:
df = pd.read_parquet("/Users/ronitguptaaa/Documents/FormulaOne/MartData/CircuitDetails")

columns = ", ".join([f"`{col}` VARCHAR(255)" for col in df.columns])

drop_table_query = f"DROP TABLE IF EXISTS CircuitDetails;"
cursor.execute(drop_table_query)
create_table_query = f"CREATE TABLE IF NOT EXISTS CircuitDetails ({columns});"
cursor.execute(create_table_query)

for index, row in df.iterrows():
    values = tuple(row.values)
    insert_query = f"REPLACE INTO CircuitDetails VALUES {values};"
    cursor.execute(insert_query)

conn.commit()

In [36]:
df = pd.read_parquet("/Users/ronitguptaaa/Documents/FormulaOne/MartData/CircuitDetails")

columns = ", ".join([f"`{col}` VARCHAR(255)" for col in df.columns])

drop_table_query = f"DROP VIEW IF EXISTS vw_CircuitDetails;"
cursor.execute(drop_table_query)
create_table_query = f"CREATE VIEW vw_CircuitDetails AS (SELECT * FROM CircuitDetails);"
cursor.execute(create_table_query)

conn.commit()

Qualifying Details

In [37]:
df = spark.sql(
    """SELECT 
        CAST(qualifyid AS INT) AS QualifyID,
        CAST(raceid AS INT) AS RaceID,
        CAST(driverid AS INT) AS DriverID,
        CAST(constructorid AS INT) AS ConstructorID,
        CAST(position AS INT) AS Position,
        CAST(q1 AS STRING) AS Q1,
        CAST(q2 AS STRING) AS Q2,
        CAST(q3 AS STRING) AS Q3,
        CONCAT(RaceID,'-',DriverID,'-',ConstructorID) AS RaceDriverConstructorID
    FROM PARQUET.`/Users/ronitguptaaa/Documents/FormulaOne/StagingData/qualifying`"""
)

df.write.mode('overwrite').format('parquet').options(header=True).save('MartData/QualifyingDetails')


In [38]:
df = pd.read_parquet("/Users/ronitguptaaa/Documents/FormulaOne/MartData/QualifyingDetails")

df.fillna(value=np.nan, inplace=True)

df.replace({np.nan: "NULL"}, inplace=True)

columns = ", ".join([f"`{col}` VARCHAR(255)" for col in df.columns])

drop_table_query = f"DROP TABLE IF EXISTS QualifyingDetails;"
cursor.execute(drop_table_query)
create_table_query = f"CREATE TABLE IF NOT EXISTS QualifyingDetails ({columns});"
cursor.execute(create_table_query)

for index, row in df.iterrows():
    values = tuple(row.values.astype(str))
    insert_query = f"INSERT INTO QualifyingDetails VALUES {values};"

conn.commit()

In [39]:
df = pd.read_parquet("/Users/ronitguptaaa/Documents/FormulaOne/MartData/QualifyingDetails")

df.fillna(value=np.nan, inplace=True)

df.replace({np.nan: "NULL"}, inplace=True)

columns = ", ".join([f"`{col}` VARCHAR(255)" for col in df.columns])

drop_table_query = f"DROP VIEW IF EXISTS vw_QualifyingDetails;"
cursor.execute(drop_table_query)
create_table_query = f"CREATE VIEW vw_QualifyingDetails AS (SELECT * FROM QualifyingDetails);"
cursor.execute(create_table_query)

conn.commit()

Lap Time Details

In [40]:
lap_times_df = spark.read.parquet('/Users/ronitguptaaa/Documents/FormulaOne/StagingData/lap_times', header=True)

total_time_df = lap_times_df.groupBy("driverid", "raceid") \
    .agg(sum("milliseconds").alias("total_milliseconds"))

total_time_df = total_time_df.withColumn("total_seconds", total_time_df["total_milliseconds"] / 1000)

total_time_df = total_time_df.withColumn("hours", total_time_df["total_seconds"] / 3600) \
    .withColumn("minutes", (total_time_df["total_seconds"] % 3600) / 60) \
    .withColumn("seconds", total_time_df["total_seconds"] % 60)

total_time_df = total_time_df.withColumn("total_time", expr("concat_ws(':', lpad(int(hours), 2, '0'), lpad(int(minutes), 2, '0'), lpad(int(seconds), 2, '0'))"))

total_time_df = total_time_df.drop("total_milliseconds", "total_seconds", "hours", "minutes", "seconds")

result_df = lap_times_df.join(total_time_df, ["driverid", "raceid"], "left")

result_df = result_df.select("raceid", "driverid", "total_time")
result_df = result_df.withColumn("RaceDriverID", concat(col("raceid"), lit("-"), col("driverid")))
result_df.write.mode('overwrite').format('parquet').options(header=True).save('MartData/LapTimeData')


In [41]:
df = pd.read_parquet("/Users/ronitguptaaa/Documents/FormulaOne/MartData/LapTimeData")

columns = ", ".join([f"`{col}` VARCHAR(255)" for col in df.columns])
drop_table_query = f"DROP TABLE IF EXISTS LapTimeData;"
cursor.execute(drop_table_query)
create_table_query = f"CREATE TABLE IF NOT EXISTS LapTimeData ({columns});"
cursor.execute(create_table_query)

for index, row in df.iterrows():
    values = tuple(row.values)
    insert_query = f"REPLACE INTO LapTimeData VALUES {values};"
    cursor.execute(insert_query)

conn.commit()

In [42]:
df = pd.read_parquet("/Users/ronitguptaaa/Documents/FormulaOne/MartData/LapTimeData")

columns = ", ".join([f"`{col}` VARCHAR(255)" for col in df.columns])
drop_table_query = f"DROP VIEW IF EXISTS vw_LapTimeData;"
cursor.execute(drop_table_query)
create_table_query = f"CREATE VIEW vw_LapTimeData AS (SELECT * FROM LapTimeData);"
cursor.execute(create_table_query)

conn.commit()